In [5]:
import pandas as pd
import string

class Slovenian_preprocessor():

    def __init__(self, data):
        self.data = data
    
    def preprocess(self):
        self.data['label'] = self.data['label'].astype(str)
        text = ''
        overall_entities = []
        entities = []
        current_entity = ''
        prev_word=''
        for _, row in self.data.iterrows():
            # print(f"word: {row['word']}, label: {row['label']}, prev_word: {prev_word}")
            if prev_word == '.' and len(text.split())>5:
                if current_entity:
                    entities.append({'entity': current_entity})
                    current_entity = ''
                # print(f"ENTRO NELLA COND")
                overall_entities.append({'text': text, 'entities': entities})
                text = ''
                entities = []
            word, label = row['word'], row['label']
            if text!='' or (word.strip() in string.punctuation) or (prev_word.strip() in string.punctuation):
                space = ' '
            else:
                space = ''
            text = text + space + word
            if label != 'O':
                if label.startswith('B-'):
                    if current_entity:
                        entities.append({'entity': current_entity})
                    current_entity = word
                elif label.startswith('I-') and current_entity:
                    current_entity += ' ' + word
            prev_word = word
        data_df = pd.DataFrame(columns=['text', 'entities'])
        for el in overall_entities:
            el['text'] = el['text'].strip() 
            data_df = pd.concat([data_df, pd.DataFrame([el])], ignore_index=True)
        self.data = data_df

In [7]:
data_train = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Train_SL_L1.csv', header=None, names=['word', 'label'])
preprocessor = Slovenian_preprocessor(data_train)
preprocessor.preprocess()
preprocessor.data

data_test = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Test_SL_L1.csv', header=None, names=['word', 'label'])
preprocessor = Slovenian_preprocessor(data_test)
preprocessor.preprocess()
preprocessor.data

,text,entities
0,35 - letni duševno bolan moški je bil premešče...,[]
1,utrpel je poškodbe polne in delne debeline poš...,[]
2,pacient je bil hospitaliziran za 66 dni in je ...,[]
3,psihiatrična diagnoza je bila shizofrenija tip...,[{'entity': 'shizofrenija'}]
4,pacient je bil občasno omejen zaradi uporniške...,[]
...,...,...
850,wagnerjev koničasti steber in streha acetabulu...,[{'entity': 'displastičnega stegnenice'}]
851,sama skodelica ecofit je povezana z zmanjšanim...,[]
852,pacient je bil zadovoljen z obsegom gibanja po...,[]
853,pacient je še vedno udoben z kolkom proteze na...,[]
